# **Importing**

In [1]:
%cd /home/ai-05/Desktop/sdp

/home/ai-05/Desktop/sdp


In [2]:
import utils

2023-04-14 20:36:04.556406: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-14 20:36:04.575989: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-14 20:36:04.705922: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-14 20:36:04.707115: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 20:36:05.626423: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
utils.set_gpu_memo_limit(5000)

GPUs: []


2023-04-14 20:36:08.851078: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-14 20:36:08.932747: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
NEWMODELNAME = "ResNet50-STBv1.0_19"
BASEMODELNAME = "ResNet50-STBv1.0_18"

# **Data generator**

In [ ]:
train_ds = ds_generator("dataset", 16, True)
test_ds = ds_generator("dcrpi-167", 16,True, shuffle=False)

# **Prep**

In [ ]:
block_layer_index = {
    0: [1,7,'conv1'],
    1: [7,19,'conv2_block1'],
    2: [19,29,'conv2_block2'],
    3: [29,39,'conv2_block3'],
    4: [39,51,'conv3_block1'],
    5: [51,61,'conv3_block2'],
    6: [61,71,'conv3_block3'],
    7: [71,81,'conv3_block4'],
    8: [81,93,'conv4_block1'],
    9: [93,103,'conv4_block2'],
    10: [103,113,'conv4_block3'],
    11: [113,123,'conv4_block4'],
    12: [123, 133,'conv4_block5'],
    13: [133,143,'conv4_block6'],
    14: [143,155,'conv5_block1'],
    15: [155,165,'conv5_block2'],
    16: [165, -1,'conv5_block3']
 }

In [ ]:
model = tf.keras.models.load_model(f"Models/{BASEMODELNAME}")

# **Tuner**

In [ ]:
tuner_path = f'Tuner/{BASEMODELNAME}'
num_trials = 30
tunable_indices = [8,9,10,11,12,13,14,15,16]

In [ ]:
def comp(hps, d_i):
    # compares two dicts if they are equal 
    for k in hps:
        if k not in d_i.index or  hps[k] != d_i[k]: return False
        else: continue
    return True

In [ ]:
def build_model(hp):
    
    resnet_base = model.layers[2]
    resnet_base.trainable = False

    name = lambda i: block_layer_index[i][2]
    names = list(map(name, tunable_indices))

    hps = {}
    for i,name_ in enumerate(names):
        hps[name_] = hp.Choice(name_, [True,False]) == True
    
    d = retrieve_trials( tuner_path, num_trials, tunable_indices)
    exists = True
    while exists:
        checks = []
        for i in range(len(d.columns)):
            equal = comp(hps,d[i])
            checks.append(equal)
            if equal: break
        checks = np.array(checks)
        exists = np.any(checks) # If there is any True value returns True
        
        if exists:
            print(f"HP combination already exists with the following values:{hps}")
        else:print(f"HP combination is new.\n{hps}")
             
        # Flip the value of the boolean at the chosen key
        flip_key = random.choice(tunable_indices)
        flip_key = name(flip_key)
        hps[flip_key] = not hps[flip_key]

    for bli_k, bli_v in block_layer_index.items():
        if bli_k in tunable_indices:
            for layer in resnet_base.layers[bli_v[0]:bli_v[1]]:
                layer.trainable = hps[name(bli_k)]
        
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=[tf.metrics.Recall()])
    return model
    

In [ ]:
from keras_tuner.engine.hyperparameters import HyperParameters

# Define the hyperparameters
hp = HyperParameters()

In [ ]:
tuner = Hyperband(
    build_model,
    objective=Objective('val_recall', direction='max'),
    max_epochs=20,
    factor=3,
    overwrite=True,
    directory=f'Tuner/{NEWMODELNAME}',
    project_name='HB_tuner')

In [ ]:
tuner.search_space_summary(extended=False)

In [ ]:
log_dir = f"logs/{NEWMODELNAME}"

# create a TensorBoard callback to visualize training metrics
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Create the TrialCheckpoint callback
class MyModelCheckpoint (ModelCheckpoint):
    def __init__(self, *args, **kwargs):
        self.now = 0
        super(MyModelCheckpoint, self).__init__(
            filepath = self.get_chpt_dir(),
            save_weights_only=False,
            save_best_only=True,
            save_freq='epoch',
            verbose=1
        )
        
    
    def on_epoch_end(self, epoch, logs=None):
        super(MyModelCheckpoint, self).on_epoch_end(epoch, logs)
        self.now = datetime.now().strftime("%Y%m%d%H%M%S")
 
    
    def get_chpt_dir(self):
        return f"Checkpoint/{NEWMODELNAME}/{self.now}_"+"{epoch:02d}.h5"

In [ ]:
tuner.search(train_ds, epochs=20, validation_data=test_ds,callbacks=[MyModelCheckpoint(), tensorboard_callback])

# **Results (Tuner)**

In [ ]:
tuner.results_summary(num_trials=100)

In [ ]:
all_hps = tuner.get_best_hyperparameters(num_trials=30)

In [ ]:
test_ds = ds_generator("dcrpi-167", 1)

In [ ]:
for i,hps in enumerate(all_hps):
    print(f"best trial - {i}")
    tuned_model = build_model(hps)
    tuned_model.evaluate(test_ds)

In [ ]:
build_model(all_hps[0]).save(f"Models/{NEWMODELNAME}")

# **Prediction**

In [ ]:
model = tf.keras.models.load_model(f"Models/{NEWMODELNAME}")

In [ ]:
test_ds = ds_generator(path="dcrpi-167",batch_size=1,to_1_hot= False,shuffle=False)
preds = model.predict(test_ds)

In [ ]:
y_preds = np.argmax(preds, axis=1)

Y = []
for images, labels in test_ds:
    for label in labels:
        Y.append(label)

y_test = np.array(Y)

In [ ]:
def show_images(i_s,images):
    nrows = 7
    ncols = 6
    class_names = ['can','paper','plastic']
    plt.figure(figsize=(14, 14))
    # for images, labels in train_ds.take(1):
    for i in range(len(images)):
        ax = plt.subplot(nrows, ncols, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"P-{class_names[y_preds[i_s[i]]]} A-{class_names[y_test[i_s[i]]]}")
        plt.axis("off")

In [ ]:
class_names = ['can','paper','plastic']
images = []
i_s = []
for i, (image, label) in enumerate(test_ds):
    if y_test[i] != y_preds[i]:
        im = tf.squeeze(image)
        if i==0: 
            print(im.shape)
        images.append(im)
        i_s.append(i)
show_images(i_s,images)

In [ ]:
cm = tf.math.confusion_matrix(y_test,y_preds)
cm

In [ ]:
train_ds = ds_generator("dataset", 16, True)
test_ds = ds_generator("dcrpi-167", 16, True)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=[tf.metrics.Recall() ])

log_dir = f"logs/{NEWMODELNAME}_p"
tf.keras.losses.mean_absolute_error
# create a TensorBoard callback to visualize training metricsf
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_callback = ModelCheckpoint(
    filepath = f'Tuner/{NEWMODELNAME}_p/'+'{epoch:02d}.h5',
    save_weights_only=False,
    save_best_only=True,
    save_freq='epoch',
    verbose=1)

In [ ]:
history = model.fit(  train_ds, 
validation_data=test_ds,
callbacks=[checkpoint_callback,tensorboard_callback],
epochs=50)

In [ ]:
model.layers[-1].get_config()